<a href="https://colab.research.google.com/github/Yazeedx0/GenAI/blob/main/LoRA_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch
!pip install safetensors
!pip install transformers
!pip install datasets

In [2]:
from datasets import load_dataset

dataset = load_dataset('csv', data_files='/content/Data-HU-23K.csv', split='train')

print(dataset[0])


Generating train split: 0 examples [00:00, ? examples/s]

{'Question': 'ما هو الحد الأعلى للعبء الدراسي في الفصل الصيفي الأول أو الثاني؟', 'Answer': '10 ساعات معتمدة، ولا يوجد حد أدنى.'}


In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, PreTrainedTokenizerFast
from peft import get_peft_model, LoraConfig

# Load model and tokenizer
model = GPT2LMHeadModel.from_pretrained("/content/drive/MyDrive/arabian_gpt_trained-2")
tokenizer = PreTrainedTokenizerFast.from_pretrained("/content/drive/MyDrive/arabian_gpt_trained-2")

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none"
)

model = get_peft_model(model, lora_config)

print(model)


PeftModel(
  (base_model): LoraModel(
    (model): GPT2LMHeadModel(
      (transformer): GPT2Model(
        (wte): Embedding(64002, 768)
        (wpe): Embedding(768, 768)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-11): 12 x GPT2Block(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2SdpaAttention(
              (c_attn): lora.Linear(
                (base_layer): Conv1D(nf=2304, nx=768)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2304, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lo

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/layer.py:1150: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [5]:
def preprocess_data(examples):
    questions = examples['Question']
    answers = examples['Answer']

    inputs = [question + " " + answer for question, answer in zip(questions, answers)]

    model_inputs = tokenizer(inputs, padding="max_length", truncation=True, max_length=512)
    model_inputs["labels"] = model_inputs["input_ids"]


    return model_inputs

train_dataset = dataset.map(preprocess_data, batched=True)


Map:   0%|          | 0/15135 [00:00<?, ? examples/s]

In [8]:

from transformers import Trainer, TrainingArguments, AdamW

training_args = TrainingArguments(
    output_dir="./results-3",
    evaluation_strategy="no",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=500,
    save_strategy="epoch",
    fp16=True,
    gradient_accumulation_steps=4,
    optim="adamw_torch",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
)

trainer.train()

<ipython-input-8-a3dfad3fd4dc>:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
500,0.098100
1000,0.097100


TrainOutput(global_step=1419, training_loss=0.09751295556141677, metrics={'train_runtime': 2238.151, 'train_samples_per_second': 20.287, 'train_steps_per_second': 0.634, 'total_flos': 1.190510026555392e+16, 'train_loss': 0.09751295556141677, 'epoch': 3.0})

In [9]:
from google.colab import drive

drive.mount('/content/drive')

save_path = '/content/drive/MyDrive/MyModels/Model-LoRA-3'

model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print("تم حفظ الموديل بنجاح على Google Drive في المسار:", save_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
تم حفظ الموديل بنجاح على Google Drive في المسار: /content/drive/MyDrive/MyModels/Model-LoRA-3


In [13]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 70.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.9 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [12]:
import gradio as gr
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast
import torch

model = GPT2LMHeadModel.from_pretrained("/content/drive/MyDrive/MyModels/Model-LoRA-3")
tokenizer = PreTrainedTokenizerFast.from_pretrained("/content/drive/MyDrive/MyModels/Model-LoRA-3")

tokenizer.pad_token = tokenizer.eos_token

def generate_text(prompt, temperature=1.0):
    inputs = tokenizer(prompt, return_tensors='pt', truncation=True, padding=True, max_length=512)

    with torch.no_grad():
        outputs = model.generate(
            inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            max_length=150,
            temperature=temperature,
            top_k=50,
            top_p=0.95,
            do_sample=True
        )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    if generated_text.lower().startswith(prompt.lower()):
        generated_text = generated_text[len(prompt):].strip()

    return generated_text

interface = gr.Interface(
    fn=generate_text,
    inputs=[gr.Textbox(label="Prompt"), gr.Slider(0, 2, step=0.1, value=1.0, label="Temperature")],
    outputs="text",
    title="اهلا بك في مساعد للجامعة الهاشمية تفضل!",
    description="أدخل نصًا للبدء بتوليد نصوص مشابهة بناءً على النموذج المدرب."
)

interface.launch()


ModuleNotFoundError: No module named 'gradio'